In [11]:
import pandas as pd

df = pd.read_csv('../../data/reddit_news.csv')

In [12]:
df.head()

,title,score,id,comments,created,body
0,Private Detectives Search for Bitcoin Creator ...,6,96im8g,3,1534011313,NaN
1,Cryptocurrency regulations in India: Modi govt...,4,96yiea,1,1534168686,NaN
2,Bitcoin fraudster &amp; head of BitConnect Ind...,54,98sk0f,18,1534762150,NaN
3,SEC Rejects Bitcoin ETFs,61,99rywy,9,1535063880,NaN
4,US Regulators Suspend Bitcoin and Ether Invest...,8,9ehxg7,4,1536538697,NaN


**Using our Model**

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from joblib import dump, load
import pandas as pd
import numpy as np
import itertools
import pickle
import sys
import os

np.set_printoptions(threshold=sys.maxsize)

In [135]:
bitcoin = pd.read_csv("./sentiment_analysis_retrain/bitcoin_price_change.csv")
reddit = pd.read_csv("./sentiment_analysis_retrain/reddit_sen_input.csv")
twitter = pd.read_csv("./sentiment_analysis_retrain/twitter_sen_input.csv")

bitcoin.columns = ("datetime","drop","sen")
bitcoin = bitcoin.drop("drop",1)
reddit.columns = ("reddit_text","datetime")
twitter.columns = ("datetime","twitter_text")
bitcoin['datetime'] = pd.to_datetime(bitcoin['datetime'], errors='coerce')
reddit['datetime'] = pd.to_datetime(reddit['datetime'], errors='coerce')
twitter['datetime'] = pd.to_datetime(twitter['datetime'], errors='coerce')
bitcoin['datetime'] = pd.to_datetime(bitcoin["datetime"].dt.strftime('%Y-%m-%d'))
reddit['datetime'] = pd.to_datetime(reddit["datetime"].dt.strftime('%Y-%m-%d'))
twitter['datetime'] = pd.to_datetime(twitter["datetime"].dt.strftime('%Y-%m-%d'))
merged1 = pd.merge(bitcoin, reddit, on='datetime')
merged2 = pd.merge(bitcoin, twitter, on='datetime')
merged1 = merged1.drop("datetime",1)
merged2 = merged2.drop("datetime",1)
merged1.columns = ("analysis", "text")
merged2.columns = ("analysis", "text")
data_reddit = merged1
data_twitter = merged2   
data_both = merged1.append(merged2)

text_data = []
data_labels = []

index_iteration = 0
for index, row in data_both.iterrows():
    text_data.append(str(row['text']))
    data_labels.append(row['analysis'])
if len(text_data) == len(data_labels): print("X,Y LENGTH VALID")

X,Y LENGTH VALID


In [136]:
solver_algorithm = "liblinear"
vectorizer = CountVectorizer(analyzer = 'word', lowercase = False)
features = vectorizer.fit_transform(text_data)
features_nd = features.toarray()
X_train, X_test, y_train, y_test  = train_test_split(features_nd, data_labels, train_size=0.80)
model = LogisticRegression(solver = solver_algorithm)
model = model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
y_pred = model.predict(X_test)
print(str(round(acc * 100, 3)) + "% ACCURACY")

60.222% ACCURACY


In [141]:
def get_our_sentiment(text):
    features = vectorizer.transform([text])
    features_nd = features.toarray()
    out = sent_model.predict(features_nd)
    return out[0]

In [148]:
df['title_polarity']= df['title'].apply(get_our_sentiment)

In [149]:
df.head()

,title,score,id,comments,created,body,title_polarity
0,New York,1,chkuap,0,1564042244,Defending the status of a financial innovation...,1.0
1,Dappstats will be the first project of crypto ...,1,chkud7,0,1564042264,[removed],0.0
2,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuis,0,1564042295,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0
3,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuu9,1,1564042363,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0
4,Historical tick-by-tick cryptocurrency market ...,1,chkuw2,0,1564042376,"Hi, if anyone here is in need of tick by tick ...",0.0


In [ ]:
df.to_csv('reddit_sent_ours.csv',index=False)

**Using textblob**

In [5]:
pip install -U textblob

Requirement already up-to-date: textblob in /home/anthony/anaconda3/lib/python3.7/site-packages (0.15.3)
Note: you may need to restart the kernel to use updated packages.


In [13]:
from textblob import TextBlob as tb

def get_sentiment(text):
    blob = tb(text)
    return tuple(blob.sentiment)

In [14]:
df['title_polarity'], df['title_subjectivity'] = df['title'].apply(get_sentiment).apply(pd.Series).values.T

In [15]:
df.head(10)

,title,score,id,comments,created,body,title_polarity,title_subjectivity
0,Private Detectives Search for Bitcoin Creator ...,6,96im8g,3,1534011313,NaN,0.0,0.375
1,Cryptocurrency regulations in India: Modi govt...,4,96yiea,1,1534168686,NaN,0.0,0.000
2,Bitcoin fraudster &amp; head of BitConnect Ind...,54,98sk0f,18,1534762150,NaN,0.0,0.000
3,SEC Rejects Bitcoin ETFs,61,99rywy,9,1535063880,NaN,-0.1,0.600
4,US Regulators Suspend Bitcoin and Ether Invest...,8,9ehxg7,4,1536538697,NaN,0.0,0.000
5,"Bitcoin rises above $6,300 as N.Y. approves do...",9,9eqlez,16,1536610045,NaN,0.0,0.050
6,Hackers hold Canadian towns computer system fo...,115,9fyl68,38,1536983701,NaN,0.0,0.000
7,Palestinians Are Using Bitcoin to Transact Acr...,24,9gsutw,4,1537257650,NaN,0.0,0.000
8,Warren Buffett says bitcoin is a 'delusion' an...,51,aulisu,37,1551103107,NaN,0.0,0.000
9,Bitcoin rallies 15% to its highest level since...,9,b8jr0f,12,1554214407,NaN,0.0,0.000


In [16]:
df.to_csv('../../data/news_sent.csv',index=False)